In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [4]:
# Define the containers as (multiplier, fixed_inhabitants)
containers = [
    (80, 6),   # A1
    (50, 4),   # A2
    (83, 7),   # A3
    (31, 2),   # A4
    (60, 4),   # A5
    (89, 8),   # B1
    (10, 1),   # B2
    (37, 3),   # B3
    (70, 4),   # B4
    (90, 10),  # B5
    (17, 1),   # C1
    (40, 3),   # C2
    (73, 4),   # C3
    (100,15),  # C4
    (20, 2),   # C5
    (41, 3),   # D1
    (79, 5),   # D2
    (23, 2),   # D3
    (47, 3),   # D4
    (30, 2),   # D5
]

TOTAL_EXTRA = 100
TOL = 1e-8
MAX_ITERS = 100

def extra_picks_sum(E):
    """Sum of extra picks = sum(max(M_i/E - H_i, 0))."""
    return sum(max(m/E - h, 0.0) for m, h in containers)

def find_equilibrium_E():
    """Bisection to solve extra_picks_sum(E) = TOTAL_EXTRA."""
    # E small => extra_picks_sum huge, E large => sum -> 0
    low, high = 1e-6, max(m/h for m, h in containers)*1.1
    for _ in range(MAX_ITERS):
        mid = (low + high) / 2
        s = extra_picks_sum(mid)
        if abs(s - TOTAL_EXTRA) < TOL:
            return mid
        if s > TOTAL_EXTRA:
            low = mid
        else:
            high = mid
    return (low + high) / 2

E_eq = find_equilibrium_E()
print(f"Equilibrium constant E ≈ {E_eq:.6f}\n")

extras = [max(m/E_eq - h, 0.0) for m, h in containers]
probs = [p / TOTAL_EXTRA * 100 for p in extras]

print(f"{'Idx':>3}  {'M_i':>5}  {'H_i':>4}  {'p_i':>9}  {'% pick':>7}  {'M/(H+p)':>9}")
print("-"*45)
for i, ((m, h), p, pct) in enumerate(zip(containers, extras, probs), start=1):
    total = h + p
    ratio = (m/total if total>0 else 0.0) * 10000
    print(f"{i:3d}  {m:5.0f}  {h:4.0f}  {p:9.5f}  {pct:7.3f}  {ratio:9.5f}")

# Sanity check: sum of extras ≈ TOTAL_EXTRA
print(f"\nSum of extra picks = {sum(extras):.6f} (target {TOTAL_EXTRA})")

Equilibrium constant E ≈ 5.661376

Idx    M_i   H_i        p_i   % pick    M/(H+p)
---------------------------------------------
  1     80     6    8.13084    8.131  56613.75661
  2     50     4    4.83178    4.832  56613.75661
  3     83     7    7.66075    7.661  56613.75661
  4     31     2    3.47570    3.476  56613.75661
  5     60     4    6.59813    6.598  56613.75661
  6     89     8    7.72056    7.721  56613.75661
  7     10     1    0.76636    0.766  56613.75661
  8     37     3    3.53551    3.536  56613.75661
  9     70     4    8.36449    8.364  56613.75661
 10     90    10    5.89720    5.897  56613.75661
 11     17     1    2.00280    2.003  56613.75661
 12     40     3    4.06542    4.065  56613.75661
 13     73     4    8.89439    8.894  56613.75661
 14    100    15    2.66355    2.664  56613.75661
 15     20     2    1.53271    1.533  56613.75661
 16     41     3    4.24206    4.242  56613.75661
 17     79     5    8.95421    8.954  56613.75661
 18     23     2    2

In [6]:
# Data for the 10 containers
multipliers = [80, 37, 10, 17, 31, 90, 50, 20, 73, 89]
inhabitants  = [ 6,  3,  1,  1,  2, 10,  4,  2,  4,  8]
extra_picks  = [
    16.6961770704,
     7.4969818950,
     1.8370221338,
     3.8229376275,
     6.7947686148,
    15.5331992042,
    10.1851106690,
     3.6740442676,
    16.7102615767,
    17.2494969908
]
# Observed actual percentages
actual_percentages = [
    18.178,  # container 1
     5.118,  # 2
     0.998,  # 3
     7.539,  # 4
     6.987,  # 5
    11.807,  # 6
     8.516,  # 7
     1.614,  # 8
    24.060,  # 9
    15.184   # 10
]

# 1) Compute predicted pick percentages
total_extra = sum(extra_picks)   # should be ≈100
predicted_percentages = [p / total_extra * 100 for p in extra_picks]

# 2) Compute over/under differences
differences = [
    actual - predicted
    for actual, predicted in zip(actual_percentages, predicted_percentages)
]

# 3) Print a summary table
print(f"{'Idx':>3}  {'M_i':>5}  {'H_i':>4}  {'Pred %':>8}  {'Act %':>6}  {'Diff %':>7}")
print("-" *  45 )
for i, (m, h, pred, act, diff) in enumerate(zip(
        multipliers, inhabitants,
        predicted_percentages, actual_percentages,
        differences), start=1):
    print(f"{i:3d}  {m:5d}  {h:4d}  {pred:8.3f}  {act:6.3f}  {diff:7.3f}")

# 4) Optional: Identify biggest over- and under-picks
over_idx, over_val = max(enumerate(differences, start=1), key=lambda x: x[1])
under_idx, under_val = min(enumerate(differences, start=1), key=lambda x: x[1])
print(f"\nBiggest over-pick: container {over_idx} by {over_val:.3f}%")
print(f"Biggest under-pick: container {under_idx} by {under_val:.3f}%")


Idx    M_i   H_i    Pred %   Act %   Diff %
---------------------------------------------
  1     80     6    16.696  18.178    1.482
  2     37     3     7.497   5.118   -2.379
  3     10     1     1.837   0.998   -0.839
  4     17     1     3.823   7.539    3.716
  5     31     2     6.795   6.987    0.192
  6     90    10    15.533  11.807   -3.726
  7     50     4    10.185   8.516   -1.669
  8     20     2     3.674   1.614   -2.060
  9     73     4    16.710  24.060    7.350
 10     89     8    17.249  15.184   -2.065

Biggest over-pick: container 9 by 7.350%
Biggest under-pick: container 6 by -3.726%


ok clearly the chatgpt solutions were significantly overpicked
that being 7% overpicked 73x, and 4% overpicked 17x

this time, chatgpt gives
If you want to maximize profit while minimizing cost, go with:

C3 (73×, 4 people) — Highest expected profit

B4 (70×, 4 people) — Still very strong even after 50k cost

C1 (17×, 1 person) — Best low-population pick with solid return even after 100k cost

ie these are probably overpicked by like 2-3% or so each.

In [7]:

# 1) Define all 20 suitcases as (label, multiplier M_i, fixed inhabitants H_i)
containers = [
    ('A1',  80,  6), ('A2',  50,  4), ('A3',  83,  7), ('A4',  31,  2), ('A5',  60,  4),
    ('B1',  89,  8), ('B2',  10,  1), ('B3',  37,  3), ('B4',  70,  4), ('B5',  90, 10),
    ('C1',  17,  1), ('C2',  40,  3), ('C3',  73,  4), ('C4', 100, 15), ('C5',  20,  2),
    ('D1',  41,  3), ('D2',  79,  5), ('D3',  23,  2), ('D4',  47,  3), ('D5',  30,  2),
]

TOTAL_EXTRA    = 100      # total strategists’ picks
BASE_TREASURE  = 10000    # seashells inside each suitcase

# 2) Compute equilibrium constant k
sum_M = sum(m for _, m, _ in containers)
sum_H = sum(h for _, _, h in containers)
k     = sum_M / (sum_H + TOTAL_EXTRA)

# 3) Compute predicted equilibrium extra‑picks per suitcase
predicted_picks = {
    label: (m / k - h)
    for label, m, h in containers
}

# 4) Plug in your observed “over‑picked” counts here (out of 100)
actual_picks = {
    'C3': 12.9,  # suitcase 73× (C3)
    'B4': 12.3,  # suitcase 70× (B4)
    'C1': 5,  # suitcase 17× (C1)
}

# 5) Build the actual p_i for all suitcases:
p_actual = {}
for label in predicted_picks:
    if label in actual_picks:
        p_actual[label] = actual_picks[label]
    else:
        p_actual[label] = predicted_picks[label]

# 6) Compute EV for each suitcase given p_actual
evs = {}
for label, m, h in containers:
    evs[label] = BASE_TREASURE * m / (h + p_actual[label])

# 7) Sort by descending EV and print recommendations
sorted_evs = sorted(evs.items(), key=lambda x: x[1], reverse=True)

print(f"{'Label':>4}  {'M_i':>4}  {'H_i':>3}  {'p_act':>8}  {'EV (shells)':>12}")
print("-" * 40)
for label, ev in sorted_evs:
    m, h = next((mm, hh) for (ll, mm, hh) in containers if ll == label)
    print(f"{label:>4}  {m:4d}  {h:3d}  {p_actual[label]:8.3f}  {ev:12.1f}")

best_label, best_ev = sorted_evs[0]
print(f"\n>>> Best choice: suitcase {best_label}, EV ≈ {best_ev:.1f} seashells.")


Label   M_i  H_i     p_act   EV (shells)
----------------------------------------
  A3    83    7     7.661       56613.8
  B1    89    8     7.721       56613.8
  B3    37    3     3.536       56613.8
  D1    41    3     4.242       56613.8
  A1    80    6     8.131       56613.8
  A2    50    4     4.832       56613.8
  A4    31    2     3.476       56613.8
  A5    60    4     6.598       56613.8
  B2    10    1     0.766       56613.8
  B5    90   10     5.897       56613.8
  C2    40    3     4.065       56613.8
  C4   100   15     2.664       56613.8
  C5    20    2     1.533       56613.8
  D2    79    5     8.954       56613.8
  D3    23    2     2.063       56613.8
  D4    47    3     5.302       56613.8
  D5    30    2     3.299       56613.8
  C3    73    4    12.900       43195.3
  B4    70    4    12.300       42944.8
  C1    17    1     5.000       28333.3

>>> Best choice: suitcase A3, EV ≈ 56613.8 seashells.


In [9]:
import random

# All suitcase labels
all_labels = [
    'A1','A2','A3','A4','A5',
    'B1','B2','B3','B4','B5',
    'C1','C2','C3','C4','C5',
    'D1','D2','D3','D4','D5'
]

# Those you’re excluding as “over‑picked”
over_picked = {'C3', 'B4', 'C1'}

# Build the remaining list
remaining = [lab for lab in all_labels if lab not in over_picked]

# Randomly pick two distinct suitcases
choice = random.sample(remaining, 2)

print("Selected suitcases:", choice)


Selected suitcases: ['D3', 'C2']
